<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/Evolution_Strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
import numpy as np
import pandas as pd
import time
import random
import ta
import tensorflow as tf
from net import *
from memory import *
from reward import *

In [0]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function,test_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.test_function = test_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            change = True
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population, change)
                change = False
            rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-7)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                self.test_function(self.weights,i)
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size=3):
      self.weights = [
            np.random.randn(input_size[0]*input_size[1], layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(1, layer_size),
        ]
#         self.weights = [
#             np.random.randn(input_size[0], input_size[1], layer_size),
#             np.random.randn(layer_size*input_size[0]*input_size[0], output_size),
#             np.random.randn(1, layer_size),
#         ]

    def predict(self, inputs):
        inputs = inputs.flatten()
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        return decision

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [0]:
class Agent:

    POPULATION_SIZE = 100
    SIGMA = 0.1
    LEARNING_RATE = 0.001

    def __init__(self, path, window_size, step_size ,spread=10, pip_cost=1000, los_cut=100):
        self.rand = np.random.RandomState()
        self.path = path
        self.window_size = window_size
        self.step_size = step_size
        self.spread = spread / pip_cost
        self.pip_cost = pip_cost
        self.los_cut = los_cut
        self.preproc()
        input_size = (window_size,self.df.shape[-1])
        self.model = Model(input_size,500)
        self.rewards = reward3
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.test,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision = self.model.predict(np.array(sequence))
        return np.argmax(decision[0])
      
    def preproc(self):
          self.dat = df = pd.read_csv(self.path)
          s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
          m = np.asanyarray(ta.macd(df["Close"])).reshape((-1, 1)) - np.asanyarray(ta.macd_signal(df["Close"])).reshape((-1, 1))
          trend3 = np.asanyarray(self.dat[["Close"]]) - np.asanyarray(ta.ema(self.dat["Close"],20)).reshape((-1, 1))
          cross1 = np.asanyarray(ta.ema(self.dat["Close"],20)).reshape((-1, 1)) - np.asanyarray(ta.ema(self.dat["Close"],5)).reshape((-1, 1))
          y = np.asanyarray(self.dat[["Open"]])
          x = np.concatenate([s,m,cross1], 1)

          gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
          self.x = []
          self.y = []
          for i in gen:
              self.x.extend(i[0].tolist())
              self.y.extend(i[1].tolist())
          self.x = np.asanyarray(self.x)
          self.y = np.asanyarray(self.y)

          self.df = self.x[-self.step_size::]
          self.trend = self.y[-self.step_size::]

    def get_reward(self, weights,change=True):
        self.model.weights = weights
        states = []
        pip = []
        provisional_pip = 0.0
        total_pip = 0.0
        position = 3
        if change:
          h = self.rand.randint(self.x.shape[0]-(self.step_size+1))
          self.df = self.x[h:h+self.step_size]
          self.trend = self.y[h:h+self.step_size]
        for t in range(0, len(self.trend) - 1):
            action = self.act(self.df[t])
            states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,action,position,states,self.pip_cost,self.spread,total_pip)
        return total_pip * self.pip_cost
      
    def test(self, weights,i):
        self.model.weights = weights
        states = []
        pip = []
        history = []
        h_p = []
        provisional_pip = 0.0
        total_pip = 0.0
        position = 3
        h = self.rand.randint(self.x.shape[0]-(self.step_size+1))
        self.df = self.x[h:h+self.step_size]
        self.trend = self.y[h:h+self.step_size]
        for t in range(0, len(self.trend) - 1):
            action = self.act(self.df[t])
            history.append(action)
            states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,action,position,states,self.pip_cost,self.spread,total_pip)
            h_p.append(position)
        self.pip = np.asanyarray(provisional_pip) * self.pip_cost
        self.pip = [p if p >= -self.los_cut else -self.los_cut for p in self.pip]
        self.total_pip = np.sum(self.pip)
        mean_pip = self.total_pip / (t + 1)
        trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
        self.trade = trade_accuracy
        mean_pip *= 24
        prob = self.prob(history)
        position_prob = self.prob(h_p)
      
        print("")
        print('action probability = ', prob)
        print("buy = ", position_prob[1], " sell = ", position_prob[-1])
        print('trade accuracy = ', trade_accuracy)
        print('epoch: %d, total rewards: %f, mean rewards: %f' % (i+1, float(self.total_pip), float(mean_pip)))
        
    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob
      
    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

#     def buy(self):
#         initial_money = self.initial_money
#         state = self.get_state(0)
#         starting_money = initial_money
#         states_sell = []
#         states_buy = []
#         inventory = []
#         for t in range(0, len(self.trend) - 1, self.skip):
#             action = self.act(state)
#             next_state = self.get_state(t + 1)
            
#             if action == 1 and initial_money >= self.trend[t]:
#                 inventory.append(self.trend[t])
#                 initial_money -= self.trend[t]
#                 states_buy.append(t)
#                 print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
            
#             elif action == 2 and len(inventory):
#                 bought_price = inventory.pop(0)
#                 initial_money += self.trend[t]
#                 states_sell.append(t)
#                 try:
#                     invest = ((close[t] - bought_price) / bought_price) * 100
#                 except:
#                     invest = 0
#                 print(
#                     'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
#                     % (t, close[t], invest, initial_money)
#                 )
#             state = next_state

#         invest = ((initial_money - starting_money) / starting_money) * 100
#         total_gains = initial_money - starting_money
#         return states_buy, states_sell, total_gains, invest

In [0]:
window_size = 30
step_size = 96
path = "audpred15.csv"

agent = Agent(path = path, 
              window_size = window_size,
              step_size = step_size)
agent.fit(iterations = 50000, checkpoint = 10)

action probability =  [0.3368421052631579, 0.35789473684210527, 0.3052631578947369]
buy =  0.4631578947368421  sell =  0.5368421052631579
trade accuracy =  0.31666666666666665
epoch: 10, total rewards: -838.000000, mean rewards: -211.705263
action probability =  [0.30526315789473685, 0.3684210526315789, 0.3263157894736842]
buy =  0.4421052631578947  sell =  0.5578947368421052
trade accuracy =  0.24528301886792453
epoch: 20, total rewards: -947.000000, mean rewards: -239.242105
action probability =  [0.3157894736842105, 0.3684210526315789, 0.3157894736842106]
buy =  0.5263157894736842  sell =  0.4736842105263158
trade accuracy =  0.3384615384615385
epoch: 30, total rewards: -948.000000, mean rewards: -239.494737
action probability =  [0.30526315789473685, 0.3684210526315789, 0.3263157894736842]
buy =  0.42105263157894735  sell =  0.5789473684210527
trade accuracy =  0.2608695652173913
epoch: 40, total rewards: -1054.000000, mean rewards: -266.273684
action probability =  [0.347368421052

In [0]:
import pickle
f = open('list.txt', 'wb')
list_row = [(1,1,1,111),(1,1,1),(1,1,11)]
pickle.dump(list_row, f)

In [0]:
f = open("./list.txt","rb")
list_row = pickle.load(f)

In [0]:
list_row

In [0]:
weights = [
  np.random.randn(30*2, 500),
  np.random.randn(500, 3),
  np.random.randn(1, 500),
]

def predict(inputs):
  feed = np.dot(inputs, weights[0]) + weights[-1]
  feed = feed.flatten().reshape((1,-1))
  print(feed.shape)
  decision = np.dot(feed, weights[1])
  return decision

In [0]:
# inputs = np.random.randn(30,2).flatten()
predict(inputs)